Rename this file with your name.

For the following questions, you need to use two files that I used in my lecture 8 (\lecture8\Data\). If you executed lecture 8 code successfully, you should have the required files uploaded into your Databricks. Otherwise, upload two CSV files into DBFS. Note that I had two csv files inside “/FileStore/tables/” folder: “airport_codes-e62ed.csv” and “departuredelays.csv”

Once you have the files, execute the following code block (once) to create a graph.

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *

# Set File Paths
airportsnaFilePath = "/FileStore/tables/airport_codes-e62ed.csv"
tripdelaysFilePath = "/FileStore/tables/departuredelays.csv"


airportsSchema = StructType([StructField("city",StringType(), True), \
                             StructField("state", StringType(), True),\
                             StructField("country", StringType(), True), \
                             StructField("iata", StringType(), True)])


# Obtain airports dataset
airportna = spark.read.format("csv"). \
              option("header", "true"). \
              schema(airportsSchema). \
              load(airportsnaFilePath)


# Obtain departure Delays data

departureDelaysSchema = StructType([StructField("id", IntegerType(), True), \
                             StructField("delay", IntegerType(), True),\
                             StructField("distance", IntegerType(), True), \
                             StructField("src", StringType(), True), \
                             StructField("dst", StringType(), True)])

departureDelays = spark.read.format("csv"). \
              option("header", "true"). \
              schema(departureDelaysSchema). \
              load(tripdelaysFilePath)

#joined dataframe

joinedDf = departureDelays.join(airportna, departureDelays.dst == airportna.iata, how='left')

tripVertices = airportna.withColumnRenamed("iata", "id")

tripEdges = joinedDf.select(joinedDf['id'], joinedDf['delay'], joinedDf['distance'],joinedDf['src'],joinedDf['dst'],joinedDf['city'], joinedDf['state'])

tripGraph = GraphFrame(tripVertices, tripEdges)

tripGraph.vertices.show(2)
tripGraph.edges.show(2)

+----------+-----+-------+---+
 city|state|country| id|
+----------+-----+-------+---+
Abbotsford| BC| Canada|YXX|
 Aberdeen| SD| USA|ABR|
+----------+-----+-------+---+
only showing top 2 rows

+-------+-----+--------+---+---+-------+-----+
 id|delay|distance|src|dst| city|state|
+-------+-----+--------+---+---+-------+-----+
1011245| 6| 602|ABE|ATL|Atlanta| GA|
1020600| -8| 369|ABE|DTW|Detroit| MI|
+-------+-----+--------+---+---+-------+-----+
only showing top 2 rows

Question 1: Find the number of flights that departs from SFO and arrives in ATL with a delay of 300 minutes.

Possible output: 2

In [4]:
display(tripGraph.edges.filter("src = 'SFO' and dst = 'ATL' and delay > 300"))


#What destinations tend to have significant delays departing from SFO

#display(tripGraph.edges.filter("src = 'SFO' and delay > 300"))

id,delay,distance,src,dst,city,state
1301204,329,1859,SFO,ATL,Atlanta,GA
2122356,376,1859,SFO,ATL,Atlanta,GA


Question 2: Show the top 4 routes that have the longest distance between two airports.

Possible answer: 


|src|dst|max(distance)|
|---|---|-------------|
|HNL|JFK|         4330|
|JFK|HNL|         4330|
|HNL|EWR|         4312|
|EWR|HNL|         4312|

In [6]:
display(tripGraph.edges.orderBy("distance").limit(4))
#fix me
#display(ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(5))


id,delay,distance,src,dst,city,state
3122210,0,21,IAD,DCA,Washington DC,null
2071330,38,23,HRL,BRO,Brownsville,TX
3041040,-9,27,WRG,PSG,null,null
3021040,-24,27,WRG,PSG,null,null


Question 3: Find the average delay for the flights departing from Atlanta to other airports. Show only the top 4 records based on the delay (in ascending order based on delay).

Possible output:



|src|dst|        avg(delay)|
|---|---|------------------|
|ATL|ONT|              59.4|
|ATL|TTN| 38.98275862068966|
|ATL|GRB|30.288888888888888|
|ATL|PWM|30.023529411764706|

In [8]:
from pyspark.sql.functions import avg, min, max
Delays = tripGraph.edges.groupBy().avg("delay")
display(Delays.filter("src = 'ATL'"))


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o484.filter.
: org.apache.spark.sql.AnalysisException: cannot resolve '`src`' given input columns: [avg(delay)]; line 1 pos 0;
'Filter ('src = ATL)
+- Aggregate [avg(cast(delay#2956 as bigint)) AS avg(delay)#3091]
 +- Project [id#2955, delay#2956, distance#2957, src#2958, dst#2959, city#2947, state#2948]
 +- Join LeftOuter, (dst#2959 = iata#2950)
 :- Relation[id#2955,delay#2956,distance#2957,src#2958,dst#2959] csv
 +- Relation[city#2947,state#2948,country#2949,iata#2950] csv

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsUp(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:147)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.checkAnalysis(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:117)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$a

Question 4: Find the number of incoming flights to SFO airport. 

Possible answer:


| id|inDegree|
|---|--------|
|SFO|   38988|

In [10]:
tripGraph.inDegrees("src = 'SFO'").count()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2521186036249501> in <module> 
 ----> 1 tripGraph . inDegrees ( "src = 'SFO'" ) . count ( ) 

 TypeError : 'DataFrame' object is not callable

Question 5: First, calculate shortest paths from different airports to SFO. Then filter the results so that it only contains airports (i.e., include id, city, state and distances) that do not have any connections to SFO (try size(distance) = 0). Then sort the airport based on the ID and show the first 5 airports (you are allowed to use multiple datagrames/graphframes/lines)


| id|         city|state|distances|
|---|-------------|-----|---------|
|ABR|     Aberdeen|   SD|       []|
|ACK|    Nantucket|   MA|       []|
|ACY|Atlantic City|   NJ|       []|
|AHN|       Athens|   GA|       []|
|AIA|     Alliance|   NE|       []|

Question 6: Using motifs, find the number of flights that departs from SEA and has a delay more than 500 minutes

Possible answer: 6

In [14]:
tripGraph.find("(a)-[e1]").filter("(b.id = 'SEA') and (e1.delay > 500)).count()


File "<command-2521186036249505>" , line 1 
 tripGraph.find("(a)-[e1]").filter("(b.id = 'SEA') and (e1.delay > 500)).count() 
 ^
 SyntaxError : EOL while scanning string literal